
# Data  Consolidation and Merging

This notebook demonstrates an optimized process for manipulating inventory data, including importing , optimizing , grouping, aggregation, and exporting results to a CSV file. The workflow has been streamlined for efficiency and readability.



## 1. Importing Libraries


In [154]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import gc


## 2. Data Loading



In [149]:
# Importing the Data
purchases_2016 = pd.read_csv("PurchasesFINAL12312016.csv")
purchases_2017 = pd.read_csv("2017PurchasePricesDec.csv")
invoce_purchases = pd.read_csv("InvoicePurchases12312016.csv")
sales = pd.read_csv("SalesFINAL12312016.csv")
Begin_Inv = pd.read_csv("BegInvFINAL12312016.csv")
End_Inv = pd.read_csv("EndInvFINAL12312016.csv")


## 3. Data Optimization


In [142]:
# Optimize the Memory usage
def optimize_memory_usage(df):
    # Downcast Integer
    for col in df.select_dtypes(include=["int64"]).columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    # Downcast Floats
    for col in df.select_dtypes(include=["float64"]).columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    # Convert Object cols to Category if appropriate
    for col in df.select_dtypes(include=["object"]).columns:
        unique_values = df[col].nunique()
        num_values = len(df[col])
        if unique_values / num_values < 0.5:
            df[col] = df[col].astype("category")

    return df

In [121]:
# The Orginal data memory usage before Optimization 
print(purchases_2016.info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372474 entries, 0 to 2372473
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   InventoryId     object 
 1   Store           int64  
 2   Brand           int64  
 3   Description     object 
 4   Size            object 
 5   VendorNumber    int64  
 6   VendorName      object 
 7   PONumber        int64  
 8   PODate          object 
 9   ReceivingDate   object 
 10  InvoiceDate     object 
 11  PayDate         object 
 12  PurchasePrice   float64
 13  Quantity        int64  
 14  Dollars         float64
 15  Classification  int64  
dtypes: float64(2), int64(6), object(8)
memory usage: 1.2 GB
None


In [143]:
# Optimize purchases_2016
df_optimized = optimize_memory_usage(purchases_2016)
print(df_optimized.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372474 entries, 0 to 2372473
Data columns (total 16 columns):
 #   Column          Dtype   
---  ------          -----   
 0   InventoryId     category
 1   Store           int8    
 2   Brand           int32   
 3   Description     category
 4   Size            category
 5   VendorNumber    int32   
 6   VendorName      category
 7   PONumber        int16   
 8   PODate          category
 9   ReceivingDate   category
 10  InvoiceDate     category
 11  PayDate         category
 12  PurchasePrice   float32 
 13  Quantity        int16   
 14  Dollars         float64 
 15  Classification  int8    
dtypes: category(8), float32(1), float64(1), int16(2), int32(2), int8(2)
memory usage: 121.9 MB
None


## 4. Purchases Data Transformation

In [144]:
# Calculate the Freight per Item
invoce_purchases["Freight/Unit"] = invoce_purchases["Freight"] / invoce_purchases["Quantity"]

In [145]:
# Select the columns for Merging 
invoce_purchases_selelected_cols = invoce_purchases[["PONumber","Freight/Unit"]]

In [146]:
# Drpoing VendorName and Description from purchases_2016
df_optimized =df_optimized.drop(["Description","VendorName"], axis = 1)

In [147]:
# Convert Categorical to String
df_optimized['PODate'] = df_optimized['PODate'].astype(str)
df_optimized['ReceivingDate'] = df_optimized['ReceivingDate'].astype(str)
df_optimized['InvoiceDate'] = df_optimized['InvoiceDate'].astype(str)
df_optimized['PayDate'] = df_optimized['PayDate'].astype(str)

In [106]:
# Convert Dates  to Datetime
df_optimized['PODate'] = pd.to_datetime(df_optimized['PODate'], errors='coerce')
df_optimized['ReceivingDate'] = pd.to_datetime(df_optimized['ReceivingDate'], errors='coerce')
df_optimized['InvoiceDate'] = pd.to_datetime(df_optimized['InvoiceDate'], errors='coerce')
df_optimized['PayDate'] = pd.to_datetime(df_optimized['PayDate'], errors='coerce')

## 5. Purchases and Invoices Data Merging
The datasets are merged on common keys (e.g, InventoryId) to consolidate information from multiple sources.

In [107]:
# Merging the Purchases with Invoices
Purchases = pd.merge(df_optimized,invoce_purchases_selelected_cols ,on = ["PONumber"], how = "left")

In [108]:
# Calculate the freight per quantity
Purchases["Freight"] =  Purchases["Freight/Unit"] * Purchases["Quantity"]
Purchases = Purchases.drop("Freight/Unit" ,  axis = 1)
Purchases.head()

,InventoryId,Store,Brand,Size,VendorNumber,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification,Freight
0,69_MOUNTMEND_8412,69,8412,750mL,105,8124,2015-12-21,2016-01-02,2016-01-04,2016-02-16,35.709999,6,214.26,1,3.470000
1,30_CULCHETH_5255,30,5255,1.75L,4466,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.350000,4,37.40,1,2.285333
2,34_PITMERDEN_5215,34,5215,1.75L,4466,8137,2015-12-22,2016-01-02,2016-01-07,2016-02-21,9.410000,5,47.05,1,2.856667
3,1_HARDERSFIELD_5255,1,5255,1.75L,4466,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.350000,6,56.10,1,3.428000
4,76_DONCASTER_2034,76,2034,750mL,388,8169,2015-12-24,2016-01-02,2016-01-09,2016-02-16,21.320000,5,106.60,1,4.610000


### Checking for Missing Values

In [109]:
# Check Nulls
Purchases.isna().sum()

InventoryId       0
Store             0
Brand             0
Size              3
VendorNumber      0
PONumber          0
PODate            0
ReceivingDate     0
InvoiceDate       0
PayDate           0
PurchasePrice     0
Quantity          0
Dollars           0
Classification    0
Freight           0
dtype: int64

In [110]:
# Droping Nulls from size column
Purchases = Purchases.dropna()

In [112]:
Purchases.dtypes

InventoryId             category
Store                       int8
Brand                      int32
Size                    category
VendorNumber               int32
PONumber                   int16
PODate            datetime64[ns]
ReceivingDate     datetime64[ns]
InvoiceDate       datetime64[ns]
PayDate           datetime64[ns]
PurchasePrice            float32
Quantity                   int16
Dollars                  float64
Classification              int8
Freight                  float64
dtype: object

## 6. Exporting the Processed Purchases and Invoices


In [113]:
# Exporting the new processed Data
Purchases.to_csv("PurchasesInvoices.csv", index = False)

### Merge Sales with Purchases 

In [138]:
# head od sales 
sales.head()

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/1/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,1/2/2016,750,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/3/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,1/8/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,1/9/2016,375,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [ ]:
# selected columns for merging 
sales_cols = [[]]

In [135]:
Purchases = pd.read_csv("PurchasesInvoices.csv")

...

## 7. Inventorey Data Transformation

In [32]:
# Droping some columns from End Inventorey before merging 
End_Inv = End_Inv.drop(["Description","Size","Price","endDate","City"], axis = 1)

In [33]:
# Drop StartDate from Begin Inventory
Begin_Inv = Begin_Inv.drop("startDate", axis = 1)

## 8.  Start and End Inventory Data Merging 


In [34]:
# Merge the Data on the common columns
merged_inventory = pd.merge(
    Begin_Inv,
        End_Inv,
   
    on=["Store","InventoryId","Brand"], 
    how='left',
    suffixes=('_start', '_end')
)

## 9. Inventory Data and Purchases Quantity from purchases 2016 Merging 


In [35]:
# Select Brand and Quantity from Purchases 
Cols_for_merge = purchases_2016[["Store","Quantity","InventoryId","Brand"]]

In [36]:
# Merge purchases Quantity with Inventory 
merged_inventory_Pur = pd.merge(merged_inventory,Cols_for_merge , on = ["Store","InventoryId","Brand"] , how = "left")
merged_inventory_Pur.head()

,InventoryId,Store,City,Brand,Description,Size,onHand_start,Price,onHand_end,Quantity
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0
1,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0
2,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0
3,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0
4,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,5.0


## 10.Inventory_Purchases Data and Sales Quantity from Sales Merging 


In [37]:
# Selected cols from sales for merging with inventory 
sales_cols = sales[["InventoryId","Store","SalesQuantity","Brand"]]

In [38]:
# merge sales_col with Inverntory 
Inventory_final = pd.merge(merged_inventory_Pur,
                           sales_cols,
                           on = ["Store","InventoryId","Brand"] 
                           , how = "left",
                          )

## 11. Inventory Data Transformation

In [39]:
# Change the Quantity Column name from Quantity to PurchasedQuantity
Inventory_final = Inventory_final.rename(columns={"Quantity": "PurchasedQuantity"})
Inventory_final.head()

,InventoryId,Store,City,Brand,Description,Size,onHand_start,Price,onHand_end,PurchasedQuantity,SalesQuantity
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0,1.0
1,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0,2.0
2,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0,1.0
3,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0,1.0
4,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,11.0,6.0,1.0


In [40]:
# Droping description and size 
Inventory_final = Inventory_final.drop(["Size","Description"], axis = 1)


## 3. Grouping and Aggregation

We group the data by `Store`, `InventoryId`, and `Brand`, and apply aggregation functions to calculate sums for various columns. This step consolidates the inventory data by these groupings.


In [41]:

# Grouping data and calculating sums for specified columns
grouped_data = Inventory_final.groupby(
    ['Store', 'InventoryId', 'Brand']
).agg({
    'onHand_start': 'sum',
    'Price': 'sum',
    'onHand_end': 'sum',
    'PurchasedQuantity': 'sum',
    'SalesQuantity': 'sum'
}).reset_index()

# Displaying the first 5 rows of the grouped data
grouped_data.head()


,Store,InventoryId,Brand,onHand_start,Price,onHand_end,PurchasedQuantity,SalesQuantity
0,1,1_HARDERSFIELD_1000,1000,0,14.99,0.0,0.0,0.0
1,1,1_HARDERSFIELD_1001,1001,0,11.98,0.0,120.0,0.0
2,1,1_HARDERSFIELD_10021,10021,50,23.98,46.0,24.0,12.0
3,1,1_HARDERSFIELD_1004,1004,221,214.37,0.0,234.0,17.0
4,1,1_HARDERSFIELD_1005,1005,70,349.90,0.0,60.0,11.0



## 4. Data Integrity Checks


In [42]:

# Checking the shape 
grouped_data.shape


(206529, 8)


### Checking for Missing Values

In [43]:

# Checking for null values 
grouped_data.isna().sum()


Store                0
InventoryId          0
Brand                0
onHand_start         0
Price                0
onHand_end           0
PurchasedQuantity    0
SalesQuantity        0
dtype: int64


## 5. Exporting the Inventory Data

In [44]:

# Exporting the Inventory Data to a CSV file
grouped_data.to_csv("Inventory.csv", index=False)
